In [1]:
import os
os.chdir("../.")

# Rectangular Section

1. [Importer la classe **RectangularSection**](#1---Importer-la-classe-RectangularSection)
2. [Créer une section rectangulaire](#2---Créer-une-section-rectangulaire)
3. [Modifier les paramètres de définition](#3---Modifier-les-paramètres-de-définition)
4. [Calculer la hauteur d'eau **h**](#4---Calculer-la-hauteur-d'eau-h)
    - [Régime critique](#h-en-régime-critique)
    - [Loi de frottement de Manning-Strickler](#h-avec-la-loi-de-frottement-de-Manning-Strickler)
    - [Loi de frottement de Ferguson](#h-avec-la-loi-de-frottement-de-Ferguson)
5. [Calculer le débit **Q**](#5---Calculer-le-débit-Q)
    - [Régime critique](#Q-en-régime-critique)
    - [Loi de frottement de Manning-Strickler](#Q-avec-la-loi-de-frottement-de-Manning-Strickler)
    - [Loi de frottement de Ferguson](#Q-avec-la-loi-de-frottement-de-Ferguson)
6. [Accéder aux paramètres géométriques et physiques](#6---Accéder-aux-paramètres-géométriques-et-physiques)
7. [Afficher un résumé de la section](#7---Afficher-un-résumé-de-la-section)
8. [A retenir](#8---A-retenir)

## 1 - Importer la classe **RectangularSection**

In [2]:
from openchannel.rectangularSection import RectangularSection

## 2 - Créer une section rectangulaire

In [3]:
rect_section = RectangularSection()
rect_section

Rectangular Section | b = 1.0 m

## 3 - Modifier les paramètres de définition

In [4]:
rect_section.b = 3.0
rect_section.name = "section rectangulaire"

In [5]:
rect_section

section rectangulaire | b = 3.0 m

## 4 - Calculer la hauteur **h**

Plusieurs méthodes:
- [Régime critique](#h-en-régime-critique)
- [Loi de frottement de Manning-Strickler](#h-avec-la-loi-de-frottement-de-Manning-Strickler)
- [Loi de frottement de Ferguson](#h-avec-la-loi-de-frottement-de-Ferguson)

### **h** en régime critique

```python
h = rect_section.get_h(Q=..., law="critical")
```

**Entrée:**
- **$Q$**: Discharge in $m^{3}.s^{-1}$

**Sortie:**
- $h$: Water Height in $m$

Avec l'hypothèse d'un **régime critique**:

<center> $Fr = 1$ </center>
<center> $\Leftrightarrow \frac{v}{\sqrt{g.h}} = 1$ </center>
<center> $\Leftrightarrow v^2 = g \times h$ </center>
<center> $\Leftrightarrow \left( \frac{Q}{A} \right)^2 = g \times h$ </center>
<center> $\Leftrightarrow \left( \frac{Q}{b h} \right)^2 = g \times h$ </center>
<center> $\Leftrightarrow \boxed{h = \left( \frac{Q}{\sqrt{g} b} \right)^{\frac{2}{3}}}$ </center>

In [6]:
rect_section.get_h(Q=20., law="critical")

1.6546881555053683

### **h** avec la loi de frottement de Manning-Strickler

```python
h = rect_section.get_h(Q=..., law="manning-strickler", Ks=..., Sb=..., h0=...)
```

**Entrées:**
- $Q$: Discharge in $m^{3}.s^{-1}$
- $K_{s}$: Strickler coefficient in $m^{\frac{1}{3}}.s^{-1}$
- $S_b$: Bed Slope in $m.m^{-1}$
- $h_{0}$: Starting estimate in $m$

**Sortie:**
- $h$: Water Height in $m$

Avec la loi de frottement de **Manning-Strickler** et à l'état d'équilibre:

<center> $v = K_{s} \times \sqrt{R S_{b}}$ </center>
<center> $\Leftrightarrow v \times A = K_{s} \times \sqrt{R S_{b}} \times A$ </center>
<center> $\Leftrightarrow Q = K_{s} \times \sqrt{R S_{b}} \times A$ </center>
<center> $\Leftrightarrow Q = K_{s} \times \sqrt{S_{b}} \times \sqrt{\frac{b h}{b + 2 h}} \times b \times h$ </center>

<center> $\Leftrightarrow \boxed{Q = K_{s} \times \frac{(b h)^\frac{3}{2}}{\sqrt{b + 2 h}} \times \sqrt{S_{b}}}$ </center>

Ici, il n'est pas possible d'exprimer simplement $h$ en fonction de $Q$. L'estimation de $h$ passe donc par des méthodes numériques.  
On définit ainsi:

<center> $f(h) = K_{s} \times \frac{(b h)^\frac{3}{2}}{\sqrt{b + 2 h}} \times \sqrt{S_{b}} - Q$ </center>

On estime numériquement $h$, tel que:

<center> $f(h) = 0$ </center>

On définit également la dérivée première de cette fonction:

<center> $f'(h) = K_{s} \times \sqrt{S_{b}} \times \frac{\frac{3}{2}b^{\frac{3}{2}} \sqrt{h} (b + 2 h) - (b h)^\frac{3}{2}}{(b + 2 h)^\frac{3}{2}}$ </center>

In [7]:
rect_section.get_h(Q=20, law="manning-strickler", Ks=25, Sb=0.1, h0=0.5)

1.0677406792325084

### **h** avec la loi de frottement de Ferguson

```python
h = rect_section.get_h(Q=..., law="ferguson", d84=..., Sb=...)
```

**Entrées:**
- $Q$: Discharge in $m^3.s^{-1}$
- $d_{84}$: Diameter in $m$
- $S_b$: Bed Slope in $m.m^{-1}$

**Sortie:**
- $h$: Water Height in $m$

D'après [Rickenmann and Recking, 2011]:

<center> $\frac{v}{\sqrt{g.S_{b}.d_{84}}} = 1.443 \times \hat{q}^{0.6} \times \left[ 1 + \left( \frac{\hat{q}}{43.78} \right)^{0.8214} \right]^{-0.2435}$ </center>

avec

<center> $\hat{q} = \frac{q}{\sqrt{g.S_{b}.d_{84}^{3}}}$ </center>

et

<center> $q = \frac{Q}{b}$ </center>

il vient:

<center> $\boxed{v = 1.443 \times \sqrt{g.S_{b}.d_{84}} \times \left( \frac{Q}{b.\sqrt{g.S_{b}.d_{84}^3}} \right)^{0.6} \times \left[ 1 + \left( \frac{Q}{43.78.b.\sqrt{g.S_{b}.d_{84}^3}} \right)^{0.8214} \right]^{-0.2435}}$</center>

Enfin:

<center> $v = \frac{Q}{A}$ </center>
<center> $\Leftrightarrow v = \frac{Q}{b.h}$ </center>
<center> $\Leftrightarrow \boxed{h = \frac{Q}{v.b}}$ </center>

In [8]:
rect_section.get_h(Q=20., law="ferguson", d84=0.4, Sb=0.1)

1.1658823343352798

## 5 - Calculer le débit **Q**

Plusieurs méthodes:
- [Régime critique](#Q-en-régime-critique)
- [Loi de frottement de Manning-Strickler](#Q-avec-la-loi-de-frottement-de-Manning-Strickler)
- [Loi de frottement de Ferguson](#Q-avec-la-loi-de-frottement-de-Ferguson)

### **Q** en régime critique

```python
Q = rect_section.get_Q(h=..., law="critical")
```

**Entrée:**
- $h$ : Water Height in $m$

**Sortie:**
- $Q$: Discharge in $m^3.s^{-1}$

Avec l'hypothèse d'un **régime critique**:

<center> $Fr = 1$ </center>
<center> $\Leftrightarrow \frac{v}{\sqrt{g.h}} = 1$ </center>
<center> $\Leftrightarrow v^2 = g \times h$ </center>
<center> $\Leftrightarrow \left( \frac{Q}{A} \right)^2 = g \times h$ </center>
<center> $\Leftrightarrow \left( \frac{Q}{b.h} \right)^2 = g \times h$ </center>
<center> $\Leftrightarrow \boxed{Q = b \times h^{\frac{3}{2}} \times \sqrt{g}}$ </center>

In [9]:
rect_section.get_Q(h=1.6546881555053683, law="critical")

19.999999999999996

### **Q** avec la loi de frottement de Manning-Strickler

```python
Q = rect_section.get_Q(h=..., law="manning-strickler", Ks=..., Sb=...)
```

**Entrées:**
- $h$ : Water Height in $m$
- $K_{s}$: Strickler coefficient in $m^{\frac{1}{3}}.s^{-1}$
- $S_b$: Bed Slope in $m.m^{-1}$

**Sortie:**
- $Q$: Discharge in $m^3.s^{-1}$

Avec la loi de frottement de **Manning-Strickler** et à l'état d'équilibre:

<center> $v = K_{s} \times \sqrt{R.S_{b}}$ </center>
<center> $\Leftrightarrow v \times A = K_{s} \times \sqrt{R.S_{b}} \times A$ </center>
<center> $\Leftrightarrow Q = K_{s} \times \sqrt{R.S_{b}} \times A$ </center>
<center> $\Leftrightarrow Q = K_{s} \times \sqrt{S_{b}} \times \sqrt{\frac{b.h}{b + 2.h}} \times b \times h$ </center>
<center> $\Leftrightarrow \boxed{Q = K_{s} \times \frac{(b.h)^\frac{3}{2}}{\sqrt{b + 2.h}} \sqrt{S_{b}}}$ </center>

In [10]:
rect_section.get_Q(h=1.0677406792325084, law="manning-strickler", Ks=25, Sb=0.1)

20.000000000000004

### **Q** avec la loi de frottement de Ferguson

```python
Q = rect_section.get_Q(h=..., law="ferguson", d84=..., Sb=..., Q0=...)
```

**Entrées:**
- $h$ : Water Height in $m$
- $d_{84}$: Diameter in $m$
- $S_b$: Bed Slope in $m.m^{-1}$
- $Q_0$: Starting estimate in $m^3.s^{-1}$

**Sortie:**
- $Q$: Discharge in $m^3.s^{-1}$

D'après [Rickenmann and Recking, 2011]:

<center> $\frac{v}{\sqrt{g.S_{b}.d_{84}}} = 1.443 \times \hat{q}^{0.6} \times \left[ 1 + \left( \frac{\hat{q}}{43.78} \right)^{0.8214} \right]^{-0.2435}$ </center>

avec

<center> $\hat{q} = \frac{q}{\sqrt{g.S_{b}.d_{84}^{3}}}$ </center>

et

<center> $q = \frac{Q}{b}$ </center>

il vient:

<center> $v = 1.443 \times \sqrt{g.S_{b}.d_{84}} \times \left( \frac{Q}{b.\sqrt{g.S_{b}.d_{84}^3}} \right)^{0.6} \times \left[ 1 + \left( \frac{Q}{43.78.b.\sqrt{g.S_{b}.d_{84}^3}} \right)^{0.8214} \right]^{-0.2435}$</center>

Enfin:

<center> $v = \frac{Q}{A}$ </center>
<center> $\Leftrightarrow v = \frac{Q}{b.h}$ </center>
<center> $\Leftrightarrow Q = v \times h \times b$ </center>
<center> $\boxed{Q = 1.443 \times h \times b \sqrt{g.S_{b}.d_{84}} \times \left( \frac{Q}{b.\sqrt{g.S_{b}.d_{84}^3}} \right)^{0.6} \times \left[ 1 + \left( \frac{Q}{43.78.b.\sqrt{g.S_{b}.d_{84}^3}} \right)^{0.8214} \right]^{-0.2435}}$ </center>

Ici, il n'est pas possible d'exprimer simplement $Q$ en fonction de $h$. L'estimation de $Q$ passe donc par des méthodes numériques.  
On définit ainsi:

<center> $f(h) = 1.443 \times h \times b \sqrt{g.S_{b}.d_{84}} \times \left( \frac{Q}{b.\sqrt{g.S_{b}.d_{84}^3}} \right)^{0.6} \times \left[ 1 + \left( \frac{Q}{43.78.b.\sqrt{g.S_{b}.d_{84}^3}} \right)^{0.8214} \right]^{-0.2435} - Q$ </center>

On estime numériquement $Q$, tel que:

<center> $f(Q) = 0$ </center>

On définit également la dérivée première de cette fonction:

<center> $f'(Q) = -1 + \frac{0.8658.b.h.\left(\frac{Q}{b.\sqrt{S_{b}.d_{84}^{3}.g}}\right)^{0.6}.\sqrt{S_{b}.d_{84}.g}}{Q.\left(0.044859671915054.\left(\frac{Q}{b.\sqrt{S_{b}.d_{84}^{3}.g}}\right)^{0.8214} + 1\right)^{0.2435}} - \frac{0.0129472068990062.b.h. \left(\frac{Q}{b.\sqrt{S_{b}.d_{84}^{3}.g}}\right)^{1.4214}.\sqrt{S_{b}.d_{84}.g}}{Q.\left(0.044859671915054.\left(\frac{Q}{b.\sqrt{S_{b}.d_{84}^{3}.g}}\right)^{0.8214} + 1\right)^{1.2435}}$ </center>

In [11]:
rect_section.get_Q(h=1.1658823343352798, law="ferguson", d84=0.40, Sb=0.1, Q0=10)

20.005242002574896

## 6 - Accéder aux paramètres géométriques et physiques

<center>
    
|    Paramètre               |          Commande                    | Formule                |
|      :---                  |              ----:                   |  :-----:               |
| Largeur de base $b$        | `base_width = rect_section.b`        | $b$                    |
| Hauteur d'eau $h$          | `water_height = rect_section.h`      | $h$                    |
| Débit $Q$                  | `discharge = rect_section.Q`         | $Q$                    |
| Largeur au Miroir $T$      | `top_width = rect_section.T`         | $b$                    |
| Surface Mouillée $A$       | `area = rect_section.A`              | $b.h$                  |
| Périmètre Mouillé $P$      | `wetted_perimeter = rect_section.P`  | $b+2.h$                |
| Rayon Hydraulique $R$      | `hydraulic_radius = rect_section.R`  | $\frac{b.h}{b + 2.h}$  |
| Profondeur Hydraulique $D$ | `hydraulic_depth = rect_section.D`   | $h$                    |
| Charge $H$                 | `energy = rect_section.H`            | $h + \frac{v^2}{2.g}$  |
| Vitesse $v$                | `velocity = rect_section.v`          | $\frac{Q}{b.h}$        |
| Nombre de Froude $Fr$      | `froude_number = rect_section.Fr`    | $\frac{v}{\sqrt{g.h}}$ |

</center>

## 7 - Afficher un résumé de la section

In [12]:
rect_section.name = str(rect_section)
rect_section.note = "---"

In [13]:
rect_section.summary()

+--------------------------------------------------------------------------------+
|                       section rectangulaire | b = 3.0 m                        |
|                                      ---                                       |
|                                 law : ferguson                                 |
+================================================================================+
|     Base Width     |     Largeur de Base      |           b = 3.0 m            |
+--------------------+--------------------------+--------------------------------+
|    Water Height    |      Hauteur d'Eau       |          h = 1.166 m           |
+--------------------+--------------------------+--------------------------------+
|     Discharge      |          Débit           |        Q = 20.005 m^3/s        |
+--------------------+--------------------------+--------------------------------+
|     Top Width      |    Largeur au Miroir     |           T = 3.0 m            |
+---

## 8 - A retenir

```python
# importer la classe RectangularSection #
# ------------------------------------- #
from openchannel.rectangularSection import RectangularSection
```

```python
# créer une section rectangulaire #
# ------------------------------- #
rect_section = RectangularSection()
```

```python
# éditer la géométrie #
# ------------------- #
rect_section.b = ...

```

```python
# déterminer la hauteur d'eau... #
# ------------------------------ #
# ...en régime critique
h = rect_section.get_h(Q=..., law="critical")

# ...avec la loi de frottement de Manning-Strickler
h = rect_section.get_h(Q=..., law="manning-strickler", Ks=..., Sb=..., h0=...)

# ...avec la loi de frottement de Ferguson
h = rect_section.get_h(Q=..., law="ferguson", d84=..., Sb=...)
```

```python
# déterminer le débit... #
# ---------------------- #
# ...en régime critique
Q = rect_section.get_Q(h=..., law="critical")

# ...avec la loi de frottement de Manning-Strickler
Q = rect_section.get_Q(h=..., law="manning-strickler", Ks=..., Sb=...)

# ...avec la loi de frottement de Ferguson
Q = rect_section.get_Q(h=..., law="ferguson", d84=..., Sb=..., Q0=...)

```

```python
# accéder aux paramètres géométriques et physiques #
# ------------------------------------------------ #
base_width = rect_section.b       # Largeur de base
water_height = rect_section.h     # Hauteur d'eau
discharge = rect_section.Q        # Débit
top_width = rect_section.T        # Largeur au Miroir
area = rect_section.A             # Surface Mouillée
wetted_perimeter = rect_section.P # Périmètre Mouillé
hydraulic_radius = rect_section.R # Rayon Hydraulique
hydraulic_depth = rect_section.D  # Profondeur Hydraulique
energy = rect_section.H           # Charge
velocity = rect_section.v         # Vitesse
froude_number = rect_section.Fr   # Nombre de Froude
```

```python
# afficher un résumé #
# ------------------ #
rect_section.name = "..."
rect_section.note = "..."
rect_section.summary()
```